In [26]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

batch_size = 128
num_classes = 10
epochs = 5

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32')
x_test = x_test.reshape(10000, 784).astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, input_shape=(784, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_split=0.1,
                 callbacks=None)

# Slackへ投稿するCallback
hostname = os.uname()[1]
print(hist.history['acc'])
print(hostname)


slack_command = 'curl -X POST -H \'Content-type: application/json\' --data \'{{"text":"Here is {}.\nepoch:{:03d}, loss:{:.3f}, val_loss:{:.3f}, acc:{:.3f}, val_acc:{:.3f}"}}\' https://hooks.slack.com/services/<own API key>'
slack_command = slack_command.format(
        hostname,
        hist.epoch[-1],
        hist.history['loss'][-1], hist.history['val_loss'][-1],
        hist.history['acc'][-1], hist.history['val_acc'][-1])
os.system(slack_command)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 5s - loss: 0.2632 - acc: 0.9182 - val_loss: 0.0953 - val_acc: 0.9713
Epoch 2/5
54000/54000 [==============================] - 5s - loss: 0.1082 - acc: 0.9680 - val_loss: 0.0798 - val_acc: 0.9758
Epoch 3/5
54000/54000 [==============================] - 5s - loss: 0.0769 - acc: 0.9769 - val_loss: 0.0734 - val_acc: 0.9778
Epoch 4/5
54000/54000 [==============================] - 5s - loss: 0.0627 - acc: 0.9809 - val_loss: 0.0700 - val_acc: 0.9798
Epoch 5/5
54000/54000 [==============================] - 5s - loss: 0.0509 - acc: 0.9847 - val_loss: 0.0706 - val_acc: 0.9810
[0.91822222220456162, 0.96796296299828422, 0.97687037038803104, 0.98092592596124717, 0.98474074075840135]
o-05349-mac.local


0